In [174]:
import numpy as np
import pandas as pd
from nltk import sent_tokenize
import re
import pymorphy2
import string
from tqdm import tqdm_notebook
from sklearn.metrics.pairwise import cosine_similarity
import operator
from collections import Counter
regex = re.compile('[%s]' % re.escape(string.punctuation))

In [72]:
morph = pymorphy2.MorphAnalyzer()

In [90]:
facts = ['Калифорнийский посёлок был независим от США почти 3 месяца пока не пришла пора праздновать День независимости и закупать алкоголь',
        'В наполеоновской артиллерии служили три генерала из одной семьи',
        'Совершивший первое кругосветное путешествие на велосипеде ездил по России на коне']
normalized_facts = []
for i in range(len(facts)):
    tokens = facts[i].split()
    for j in range(len(tokens)):
        tokens[j] = morph.parse(tokens[j])[0].normal_form
    normalized_facts.append(' '.join(tokens))

## Normalized facts 

In [91]:
for fact in normalized_facts:
    print(fact, '\n')

калифорнийский посёлок быть независимый от сша почти 3 месяц пока не пришлый пора праздновать день независимость и закупать алкоголь 

в наполеоновский артиллерия служить три генерал из один семья 

совершить первое кругосветный путешествие на велосипед ездить по россия на кон 



In [92]:
docs = open('data.txt', encoding='utf8').read()
docs = sent_tokenize(docs)
normalized_docs = []
for i in range(len(docs)):
    docs[i] = regex.sub(' ', docs[i])
    docs[i] = docs[i][:-1]
    tokens = docs[i].split()
    for j in range(len(tokens)):
        tokens[j] = morph.parse(tokens[j])[0].normal_form
    normalized_docs.append(' '.join(tokens))

In [93]:
text = ' '.join(normalized_docs)

In [157]:
dictionary = np.unique(text.split())
dict_frame_d = pd.DataFrame(index=range(len(normalized_docs)), columns=dictionary)
dict_frame_q = pd.DataFrame(index=range(len(normalized_facts)), columns=dictionary)

In [164]:
for i in range(len(normalized_docs)):
    counter = Counter(normalized_docs[i].split())
    for key in counter.keys():
        dict_frame_d.loc[i, key] = counter[key]
        
dict_frame_q = pd.DataFrame(index=range(len(normalized_facts)), columns=dictionary)
for i in range(len(normalized_facts)):
    counter = Counter(normalized_facts[i].split())
    for key in counter.keys():
        if key in dict_frame_d.columns:
            dict_frame_q.loc[i, key] = counter[key]


In [165]:
docs_vector_space = np.array(dict_frame_d.fillna(0))
queries_vector_space = np.array(dict_frame_q.fillna(0))

In [166]:
print('docs vector model shape: ', docs_vector_space.shape)
print('queries vector model shape: ', queries_vector_space.shape)

docs vector model shape:  (177, 1139)
queries vector model shape:  (3, 1139)


# TF only

## Fact 1
Калифорнийский посёлок был независим от США почти 3 месяца пока не пришла пора праздновать День независимости и закупать алкоголь

In [167]:
range_distances = {}
distances = cosine_similarity(docs_vector_space, [queries_vector_space[0]])
for i in range(len(distances)):
    range_distances[i] = distances[i][0]
range_distances = sorted(range_distances.items(), key=operator.itemgetter(1), reverse=True)
for i in range(5):
    print(docs[range_distances[i][0]], ' --- ', range_distances[i][1])     
    print('------------------------')

К началу революции Д’Абовилю было почти лет он имел чин лагерного маршала и пользовался заслуженной известностью среди французских военных и артиллеристов  ---  0.21535276082326618
------------------------
В следующем году старатели недовольные налогами которыми обложило их деятельность государство решили от них избавиться путём отделения от США и провозглашением собственной республики  ---  0.1987615979999813
------------------------
Оно не работало несколько месяцев в году несколько месяцев в году и с по год  ---  0.1987615979999813
------------------------
Во время Ста дней активно не служил  ---  0.1951800145897066
------------------------
В телесериале Дни в Долине Смерти о независимости и скорой обратной зависимости Раф энд Рэди рассказывается в двух эпизодах  Rough and Ready    и Birthright     ---  0.18257418583505533
------------------------


## Fact 2
В наполеоновской артиллерии служили три генерала из одной семьи

In [168]:
range_distances = {}
distances = cosine_similarity(docs_vector_space, [queries_vector_space[1]])
for i in range(len(distances)):
    range_distances[i] = distances[i][0]
range_distances = sorted(range_distances.items(), key=operator.itemgetter(1), reverse=True)
for i in range(5):
    print(docs[range_distances[i][0]], ' --- ', range_distances[i][1])     
    print('------------------------')

Родился в семейном особняке в городе Ла Фер в семье артиллерийского офицера позднее генерал лейтенанта сенатора и Первого генерального инспектора артиллерии Франсуа Д’Абовиля  ---  0.39391929857916763
------------------------
В этом качестве он участвовал в осаде Йорктауна    и был одним из тех чьи действия привели к капитуляции британцев генерала Корнуоллиса  ---  0.36860489038724287
------------------------
Первый генеральный инспектор артиллерии  ---  0.35355339059327373
------------------------
Вильям Стивенс эмигрировал в Миссури в году но вскоре вернулся из за болезни жены чтобы позже всей семьёй перебраться в США  ---  0.34020690871988585
------------------------
В году Д’Абовиль был произведён в генерал лейтенанты  ---  0.33541019662496846
------------------------


## Fact 3
Совершивший первое кругосветное путешествие на велосипеде ездил по России на коне

In [169]:
range_distances = {}
distances = cosine_similarity(docs_vector_space, [queries_vector_space[2]])
for i in range(len(distances)):
    range_distances[i] = distances[i][0]
range_distances = sorted(range_distances.items(), key=operator.itemgetter(1), reverse=True)
for i in range(5):
    print(docs[range_distances[i][0]], ' --- ', range_distances[i][1])     
    print('------------------------')

Томас Стивенс  человек совершивший первое кругосветное путешествие на велосипеде проехав  миль и пробыв в пути более двух лет  с апреля по декабрь года   ---  0.48241815132442184
------------------------
Позже Томас устроился на работу в шахту в Колорадо где ему пришла идея о путешествии на велосипеде по стране  ---  0.440086229423352
------------------------
На всём протяжении своего кругосветного путешествия Стивенс отправлял свои путевые заметки в Harper’s Magazine которые позже вышли в двухтомнике с страницами Вокруг света на велосипеде  ---  0.37907125275171766
------------------------
Стивенс провёл в экспедиции месяцев взбирался на Килиманджаро охотился на крупного зверя  ---  0.3223291856101521
------------------------
Отсюда он на велосипеде отправился в восточный Китай испытывая при этом языковые трудности  ---  0.2508726030021272
------------------------


# TF * IDF

In [170]:
idf_vector = np.zeros(docs_vector_space.shape[1])
for i in range(docs_vector_space.shape[1]):
    idf_vector[i] = np.log(docs_vector_space.shape[0] / len(np.where(docs_vector_space[:,i] > 0)[0]))
docs_vector_space_idf = docs_vector_space * idf_vector

## Fact 1
Калифорнийский посёлок был независим от США почти 3 месяца пока не пришла пора праздновать День независимости и закупать алкоголь

In [171]:
range_distances = {}
distances = cosine_similarity(docs_vector_space_idf, [queries_vector_space[0]])
for i in range(len(distances)):
    range_distances[i] = distances[i][0]
range_distances = sorted(range_distances.items(), key=operator.itemgetter(1), reverse=True)
for i in range(5):
    print(docs[range_distances[i][0]], ' --- ', range_distances[i][1])     
    print('------------------------')

Оно не работало несколько месяцев в году несколько месяцев в году и с по год  ---  0.20395584782494466
------------------------
Почтовая служба США требовала от жителей для упрощения своей работы сменить название их посёлка на Раф или Рэди но те отказались  ---  0.1715937588911348
------------------------
Во время Ста дней активно не служил  ---  0.16815573579607093
------------------------
Независимое государство с апреля по июля года  ---  0.1497504868844784
------------------------
В следующем году старатели недовольные налогами которыми обложило их деятельность государство решили от них избавиться путём отделения от США и провозглашением собственной республики  ---  0.13299096320237047
------------------------


## Fact 2
В наполеоновской артиллерии служили три генерала из одной семьи

In [172]:
range_distances = {}
distances = cosine_similarity(docs_vector_space_idf, [queries_vector_space[1]])
for i in range(len(distances)):
    range_distances[i] = distances[i][0]
range_distances = sorted(range_distances.items(), key=operator.itemgetter(1), reverse=True)
for i in range(5):
    print(docs[range_distances[i][0]], ' --- ', range_distances[i][1])     
    print('------------------------')

Первый генеральный инспектор артиллерии  ---  0.2679578477327189
------------------------
Оба его сына были генералами артиллеристами Наполеоновской армии  ---  0.2586844974497135
------------------------
Семья Д’Абовилей существует до сих пор и является одной из самых многочисленных среди сохранившихся семей французской знати  ---  0.2475701214137544
------------------------
Во время Ста дней все три представителя семьи Д’Абовиль отказались активно поддерживать Наполеона  ---  0.20539528572653754
------------------------
Родился в семейном особняке в городе Ла Фер в семье артиллерийского офицера позднее генерал лейтенанта сенатора и Первого генерального инспектора артиллерии Франсуа Д’Абовиля  ---  0.19104139099874404
------------------------


## Fact 3
Совершивший первое кругосветное путешествие на велосипеде ездил по России на коне

In [173]:
range_distances = {}
distances = cosine_similarity(docs_vector_space_idf, [queries_vector_space[2]])
for i in range(len(distances)):
    range_distances[i] = distances[i][0]
range_distances = sorted(range_distances.items(), key=operator.itemgetter(1), reverse=True)
for i in range(5):
    print(docs[range_distances[i][0]], ' --- ', range_distances[i][1])     
    print('------------------------')

Томас Стивенс  человек совершивший первое кругосветное путешествие на велосипеде проехав  миль и пробыв в пути более двух лет  с апреля по декабрь года   ---  0.45447071637957215
------------------------
Позже Томас устроился на работу в шахту в Колорадо где ему пришла идея о путешествии на велосипеде по стране  ---  0.27298770597768557
------------------------
На всём протяжении своего кругосветного путешествия Стивенс отправлял свои путевые заметки в Harper’s Magazine которые позже вышли в двухтомнике с страницами Вокруг света на велосипеде  ---  0.24477349735172058
------------------------
Однако уже в конце того же года в силу наступления войск России и её союзников генерал привлекался к оборонительным работам на западе Франции  ---  0.1364746618008845
------------------------
Отсюда он на велосипеде отправился в восточный Китай испытывая при этом языковые трудности  ---  0.12711172169966983
------------------------
